<a href="https://colab.research.google.com/github/whoislimshady/neural-networks-and-deep-learning/blob/master/Better_Retrieval_via_DPR_andgptj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Make sure you have a GPU running
!nvidia-smi

Thu May 12 16:20:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    29W /  70W |   2528MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-4vqk3ivg/farm-haystack_646754fac2f44bf6843455bc10db1744
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-4vqk3ivg/farm-haystack_646754fac2f44bf6843455bc10db1744
  Resolved https://github.com/deepset-ai/haystack.git to commit 4f22942cb0799ef236f69b9986b1ad0662fc2b70
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [16]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

ValueError: ignored

In [6]:
# Let's first get some files that we want to use
doc_dir = "data/tutorial6"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt6.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt6.zip to `data/tutorial6`
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/348_The_Winds_of_Winter.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/512_Home__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/482_Petyr_Baelish.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/487_Ramsay_Bolton.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/127_The_Climb__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/515_The_Door__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/299_Rani_Mahal__TV_series_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial6/452_Fire_and_Blood__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing -  Converting data/tut

Writing Documents:   0%|          | 0/2497 [00:00<?, ?it/s]

In [7]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)
# Important:
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base
INFO - haystack.document_stores.faiss -  Updating embeddings for 2357 docs...


Updating Embedding:   0%|          | 0/2357 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/2368 [00:00<?, ? Docs/s]

In [8]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [10]:
from haystack.pipelines import  DocumentSearchPipeline

pipe = DocumentSearchPipeline(retriever)

In [11]:
prediction = pipe.run(
    query="who is the ",params={"Retriever": {"top_k": 10}}
)

In [12]:
print(prediction)

{'documents': [<Document: {'content': "\n=== Yohn Royce ===\nYohn Royce (sometimes called 'Bronze Yohn') is Lord of Runestone and head of House Royce. After the death of Lysa Arryn, Lord Yohn forms the Lords Declarant in opposition to Petyr Baelish's rule of the Vale.\nIn the HBO television adaptation, he is portrayed by Rupert Vansittart.", 'content_type': 'text', 'score': 0.645497791450016, 'meta': {'name': '30_List_of_A_Song_of_Ice_and_Fire_characters.txt', 'vector_id': '772'}, 'embedding': None, 'id': '67422e89c9fe3b2ff50f392cbf899460'}>, <Document: {'content': '\n=== Lord of Bones ===\nThe Lord of Bones (also mocked as "Rattleshirt") is a wildling leader under Mance Rayder, known for his vileness and cowardice. In \'\'A Dance with Dragons\'\', he is executed, believed to be Mance since Melisandre glamoured him to take the appearance of the condemned.\nIn the HBO television adaptation, he is portrayed by Edward Dogliani in seasons two and three, and Ross O\'Hennessy in season five.

In [71]:
import json
para = []
for data in prediction['documents']:
  jsonStr = json.dumps(data.__dict__)
  m = json.loads(jsonStr)
  para.append((m['content']))

  


  

In [78]:
para[0]

"\n=== Yohn Royce ===\nYohn Royce (sometimes called 'Bronze Yohn') is Lord of Runestone and head of House Royce. After the death of Lysa Arryn, Lord Yohn forms the Lords Declarant in opposition to Petyr Baelish's rule of the Vale.\nIn the HBO television adaptation, he is portrayed by Rupert Vansittart."

In [66]:

prompt = "Generate Answer to the Question from the Supporting Texts.\n\nQuestion: "+query+"\n\nSupporting Texts:\nSupporting Text 1: "+para[0]+"\nSupporting Text 2: "+para[1]+"\nSupporting Text 3: "+para[2]+"\nSupporting Text 4: "+para[3]+"\nSupporting Text 5: "+para[4]+"\nSupporting Text 6: "+para[5]+"\nSupporting Text 7: "+para[6]+"\n\nAnswer:"
print(prompt)
payload = {
        "context": prompt,
        "token_max_length": 200,
        "temperature": 0.7,
        "top_p": 0.95,
        "stop_sequence":"<|endoftext|>",
        "model":"Persona model",
            
    }
query = requests.post("http://35.239.8.170/generate", params=payload)
response = query.json()
a_string = response['text']
partitioned_string = a_string.partition('<|endoftext|>')


return partitioned_string[0]


"\n=== Yohn Royce ===\nYohn Royce (sometimes called 'Bronze Yohn') is Lord of Runestone and head of House Royce. After the death of Lysa Arryn, Lord Yohn forms the Lords Declarant in opposition to Petyr Baelish's rule of the Vale.\nIn the HBO television adaptation, he is portrayed by Rupert Vansittart."